In [ ]:
"""
SQL db -> JSONs -> schema -> SQL gen1
                          -> NL-SQL parser -> SQL gen2
"""

"""

NoSQL db -> JSONs -> schema -> SQL gen1 -> NoSQL gen1
                            -> NL-SQL parser -> SQL gen2 -> NoSQL gen2

"""


In [ ]:
from test_cases import NoSQLTestCases, SQLTestCases


This package provides tools for converting natural language queries into SQL and NoSQL database queries 
for retrieving results. It operates without the use of learnable parameters, neural networks, or embeddings, 
focusing instead on rule-based natural language parsing.

Features:
1. Supports natural language descriptions of queries that involve combinations of:
   - `SELECT` statements
   - Table names
   - Column names
   - `WHERE`, `GROUP BY`, and `ORDER BY` clauses
   Column names can include aggregated operations such as `min`, `max`, `median`, `mean`, `stddev`, `variance`, 
   `count`, and `distinct`.

2. Enforces rules for SQL queries:
   - When `WHERE`, `GROUP BY`, or `ORDER BY` clauses are used, the keywords **must appear** explicitly 
     in their respective clauses.
   - Multi-word column names are enclosed in backticks (e.g., `` `column name` ``).
   - Aggregation parameters with multi-word descriptions are supported using formats like `minimum-of`, 
     `standard-devia

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1002)>
/Users/pranu/Documents/Grad-Docs/Third Sem/FDM/ChatDB-new/.venv/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
tester = SQLTestCases(db_name="thrombosis_prediction")
cases = tester.t
_ = list(map(tester, cases))

####################################################################################################
NL Query :  From the examimation, what can you find as the average ANA for where Thrombosis = 1


Reconstructed Query :  Examination find average ANA where Thrombosis = <number>
Parse Tree :  (S
  (TABLE Examination)
  (CMD find)
  (AGG (AVG average))
  (COLUMN (NOMCAT ANA))
  (CLAUSE
    where
    (COLUMN (NOMCAT Thrombosis))
    (NOP =)
    (VALUE <number>)
    (CLAUSE )))
SQL :  SELECT AVG(ANA) FROM Examination WHERE Thrombosis = 1
Results :  [(639.2452830188679,)]
####################################################################################################
NL Query :  According to examinations data, what can you get as the diagnosis to be used for where sympotms = 'AMI'


Reconstructed Query :  Examination get Diagnosis where Symptoms = <string>
Parse Tree :  (S
  (TABLE Examination)
  (CMD get)
  (COLUMN (CAT Diagnosis))
  (CLAUSE
    where
    (COLUMN (CAT Symptoms))
    (NOP =)
    (VALUE <string>)
    (CLAUSE )))
SQL :  SELECT Diagnosis FROM Examination WHERE Symptoms = 'AMI'
Results :  [('MCTD, AMI',), ('SLE, RA',), ('SLE, AMI',)]
####################################################################################################
NL Query :  get me all the distinct symptoms found and their `ANA Pattern`, according to the examinations, where thrombosis = 3 grouping it by `ANA Pattern`


Reconstructed Query :  get distinct Symptoms `ANA Pattern` Examination where Thrombosis = <number> group by `ANA Pattern`
Parse Tree :  (S
  (CMD get)
  (AGG (DISTINCT distinct))
  (COLUMN (CAT Symptoms) (COLUMN (CAT `ANA Pattern`)))
  (TABLE Examination)
  (CLAUSE
    where
    (COLUMN (NOMCAT Thrombosis))
    (NOP =)
    (VALUE <number>)
    (CLAUSE group by (COLUMN (CAT `ANA Pattern`)) (CLAUSE ))))
SQL :  SELECT DISTINCT(Symptoms), `ANA Pattern` FROM Examination WHERE Thrombosis = 3 GROUP BY `ANA Pattern`
Results :  [('thrombocytepenia', 'P'), ('thrombocytopenia', 'S'), ('thrombocytopenia', 'S,D')]
####################################################################################################
NL Query :  find me all the `examination dates` from the examination records for where `aCL IgM` > 10


Reconstructed Query :  find `Examination Date` Examination where `aCL IgM` > <number>
Parse Tree :  (S
  (CMD find)
  (COLUMN (CAT `Examination Date`))
  (TABLE Examination)
  (CLAUSE
    where
    (COLUMN (NUM `aCL IgM`))
    (NOP >)
    (VALUE <number>)
    (CLAUSE )))
SQL :  SELECT `Examination Date` FROM Examination WHERE `aCL IgM` > 10
Results :  [('1995-04-24',), ('1996-03-18',), ('1998-02-02',)]
####################################################################################################
NL Query :  get me the maximum value for GOT, CRP from the laboratory records where GOT > 100 and group this by CRP.


...

Reconstructed Query :  get count RBC where T-BIL > <number> Laboratory
Parse Tree :  (S
  (CMD get)
  (AGG (COUNT count))
  (COLUMN (NUM RBC))
  (CLAUSE
    where
    (COLUMN (NUM T-BIL))
    (NOP >)
    (VALUE <number>)
    (CLAUSE ))
  (TABLE Laboratory))
SQL :  SELECT COUNT(RBC) FROM Laboratory WHERE T-BIL > 0.2
[ERROR] no such column: T
Results :  []
####################################################################################################
NL Query :  Fetch the range of UA values where CRP = 2 in Laboratory.


Reconstructed Query :  fetch RA UA where CRP = <number> Laboratory
Parse Tree :  (S
  (CMD fetch)
  (COLUMN (CAT RA) (COLUMN (NUM UA)))
  (CLAUSE
    where
    (COLUMN (CAT CRP))
    (NOP =)
    (VALUE <number>)
    (CLAUSE ))
  (TABLE Laboratory))
SQL :  SELECT RA, UA FROM Laboratory WHERE CRP = 2
Results :  [('-', 7.0), (None, 8.3), (None, 6.6)]
####################################################################################################
NL Query :  Retrieve the minimum value of ALP where ALP > 15 in Laboratory.


Reconstructed Query :  retrieve minimum ALP where ALP > <number> Laboratory
Parse Tree :  (S
  (CMD retrieve)
  (AGG (MIN minimum))
  (COLUMN (NOMCAT ALP))
  (CLAUSE
    where
    (COLUMN (NOMCAT ALP))
    (NOP >)
    (VALUE <number>)
    (CLAUSE ))
  (TABLE Laboratory))
SQL :  SELECT MIN(ALP) FROM Laboratory WHERE ALP > 15
Results :  [(22,)]
####################################################################################################
NL Query :  From the patient records, where sex of person is = 'M' get me his bitrthday


Reconstructed Query :  Patient where SEX = <string> get Birthday
Parse Tree :  (S
  (TABLE Patient)
  (CLAUSE
    where
    (COLUMN (CAT SEX))
    (NOP =)
    (VALUE <string>)
    (CLAUSE ))
  (CMD get)
  (COLUMN (CAT Birthday)))
SQL :  SELECT Birthday FROM Patient WHERE SEX = 'M'
Results :  [('1937-11-24',), ('1923-07-25',), ('1967-03-25',)]
####################################################################################################
NL Query :  Get me the id of patients where their `first date` is < '1985-10-01' and group this by diagnosis results.


Reconstructed Query :  get ID Patient where `First Date` < <string> group by Diagnosis
Parse Tree :  (S
  (CMD get)
  (COLUMN (NOMCAT ID))
  (TABLE Patient)
  (CLAUSE
    where
    (COLUMN (CAT `First Date`))
    (NOP <)
    (VALUE <string>)
    (CLAUSE group by (COLUMN (CAT Diagnosis)) (CLAUSE ))))
SQL :  SELECT ID FROM Patient WHERE `First Date` < '1985-10-01' GROUP BY Diagnosis
Results :  [(549080,), (1007813,), (1218725,)]
####################################################################################################
NL Query :  In the ecords, find diagnosis of patients grouped by their sex.
Reconstructed Query :  find Diagnosis Patient group by SEX
Parse Tree :  (S
  (CMD find)
  (COLUMN (CAT Diagnosis))
  (TABLE Patient)
  (CLAUSE group by (COLUMN (CAT SEX)) (CLAUSE )))
SQL :  SELECT Diagnosis FROM Patient GROUP BY SEX
Results :  [('SLE',), ('RA susp.',), ("Raynaud's phenomenon",)]


In [5]:
tester = NoSQLTestCases(db_name="california_schools")
cases = tester.t
_ = list(map(tester, cases))

####################################################################################################
NL Query :  Grab AvgScrMath from the satscores dataset where cds = '1100170000000'.


Reconstructed Query :  grab AvgScrMath satscores where cds = <string>
Parse Tree :  (S
  (CMD grab)
  (COLUMN (NOMCAT AvgScrMath))
  (TABLE satscores)
  (CLAUSE
    where
    (COLUMN (CAT cds))
    (NOP =)
    (VALUE <string>)
    (CLAUSE )))
SQL :  SELECT AvgScrMath FROM satscores WHERE cds = '1100170000000'
NoSQL :  satscores.aggregate([{'$match': {'cds': {'$eq': '1100170000000'}}}, {'$project': {'AvgScrMath': 1}}])
Results :  [{'_id': ObjectId('67455a282a2b11d96a794ad9'), 'AvgScrMath': 418}]
####################################################################################################
NL Query :  Pick NumTstTakr out of satscores where cname = 'Alameda'.


Reconstructed Query :  pick-out NumTstTakr satscores where cname = <string>
Parse Tree :  (S
  (CMD pick-out)
  (COLUMN (NOMCAT NumTstTakr))
  (TABLE satscores)
  (CLAUSE
    where
    (COLUMN (CAT cname))
    (NOP =)
    (VALUE <string>)
    (CLAUSE )))
SQL :  SELECT NumTstTakr FROM satscores WHERE cname = 'Alameda'
NoSQL :  satscores.aggregate([{'$match': {'cname': {'$eq': 'Alameda'}}}, {'$project': {'NumTstTakr': 1}}])
Results :  [{'_id': ObjectId('67455a282a2b11d96a794ad9'), 'NumTstTakr': 88}, {'_id': ObjectId('67455a282a2b11d96a794ada'), 'NumTstTakr': 17}, {'_id': ObjectId('67455a282a2b11d96a794adb'), 'NumTstTakr': 71}]
####################################################################################################
NL Query :  Get distinct cds values from satscores.
Reconstructed Query :  get distinct cds satscores
Parse Tree :  (S
  (CMD get)
  (AGG (DISTINCT distinct))
  (COLUMN (CAT cds))
  (CLAUSE )
  (TABLE satscores))
SQL :  SELECT DISTINCT(cds) FROM satscores 
NoSQL :  

Reconstructed Query :  retrieve unique rtype satscores where AvgScrRead > <number>
Parse Tree :  (S
  (CMD retrieve)
  (AGG (DISTINCT unique))
  (COLUMN (CAT rtype))
  (TABLE satscores)
  (CLAUSE
    where
    (COLUMN (NOMCAT AvgScrRead))
    (NOP >)
    (VALUE <number>)
    (CLAUSE )))
SQL :  SELECT DISTINCT(rtype) FROM satscores WHERE AvgScrRead > 500
NoSQL :  satscores.aggregate([{'$match': {'AvgScrRead': {'$gt': 500}}}, {'$group': {'_id': None, 'result': {'$addToSet': '$rtype'}}}])
Results :  [{'_id': None, 'result': ['S', 'D']}]
####################################################################################################
NL Query :  Pull out `School Name` from frpm where CDSCode = '01100170109835'.


Reconstructed Query :  pull `School Name` frpm where CDSCode = <string>
Parse Tree :  (S
  (CMD pull)
  (COLUMN (CAT `School Name`))
  (TABLE frpm)
  (CLAUSE
    where
    (COLUMN (CAT CDSCode))
    (NOP =)
    (VALUE <string>)
    (CLAUSE )))
SQL :  SELECT `School Name` FROM frpm WHERE CDSCode = '01100170109835'
NoSQL :  frpm.aggregate([{'$match': {'CDSCode': {'$eq': '01100170109835'}}}, {'$project': {'School Name': 1}}])
Results :  [{'_id': ObjectId('67455a282a2b11d96a7923d7'), 'School Name': 'FAME Public Charter'}]
####################################################################################################
NL Query :  Extract `Percent (%) Eligible FRPM (Ages 5-17)` from frpm where `School Name` = 'Envision Academy for Arts & Technology'.


Reconstructed Query :  extract `Percent (%) Eligible FRPM (Ages 5-17)` frpm where `School Name` = <string>
Parse Tree :  (S
  (CMD extract)
  (COLUMN (NUM `Percent (%) Eligible FRPM (Ages 5-17)`))
  (TABLE frpm)
  (CLAUSE
    where
    (COLUMN (CAT `School Name`))
    (NOP =)
    (VALUE <string>)
    (CLAUSE )))
SQL :  SELECT `Percent (%) Eligible FRPM (Ages 5-17)` FROM frpm WHERE `School Name` = 'Envision Academy for Arts '
NoSQL :  frpm.aggregate([{'$match': {'School Name': {'$eq': 'Envision Academy for Arts '}}}, {'$project': {'Percent (%) Eligible FRPM (Ages 5-17)': 1}}])
Results :  []
####################################################################################################
NL Query :  Fetch all distinct `Charter School Numbers` in frpm.
Reconstructed Query :  fetch distinct `Charter School Number` frpm
Parse Tree :  (S
  (CMD fetch)
  (AGG (DISTINCT distinct))
  (COLUMN (CAT `Charter School Number`))
  (CLAUSE )
  (TABLE frpm))
SQL :  SELECT DISTINCT(`Charter School Num

Reconstructed Query :  grab distinct `School Type` frpm where `Percent (%) Eligible Free (K-12)` > <number>
Parse Tree :  (S
  (CMD grab)
  (AGG (DISTINCT distinct))
  (COLUMN (CAT `School Type`))
  (TABLE frpm)
  (CLAUSE
    where
    (COLUMN (NUM `Percent (%) Eligible Free (K-12)`))
    (NOP >)
    (VALUE <number>)
    (CLAUSE )))
SQL :  SELECT DISTINCT(`School Type`) FROM frpm WHERE `Percent (%) Eligible Free (K-12)` > 0.5
NoSQL :  frpm.aggregate([{'$group': {'_id': None, 'result': {'$addToSet': '$School Type'}}}])
Results :  [{'_id': None, 'result': ['Juvenile Court Schools', 'District Community Day Schools', 'Opportunity Schools', 'Special Education Schools (Public)', 'Elementary Schools (Public)', 'Alternative Schools of Choice', 'Junior High Schools (Public)', 'Youth Authority Facilities', 'K-12 Schools (Public)', 'State Special Schools', 'High Schools (Public)', None, 'County Community', 'Continuation High Schools', 'Elemen Schools In 1 School Dist. (Public)', 'Preschool', 'Int

Reconstructed Query :  fetch AdmEmail1 schools where CDSCode = <string>
Parse Tree :  (S
  (CMD fetch)
  (COLUMN (CAT AdmEmail1))
  (TABLE schools)
  (CLAUSE
    where
    (COLUMN (CAT CDSCode))
    (NOP =)
    (VALUE <string>)
    (CLAUSE )))
SQL :  SELECT AdmEmail1 FROM schools WHERE CDSCode = '01100170000000'
NoSQL :  schools.aggregate([{'$match': {'CDSCode': {'$eq': '01100170000000'}}}, {'$project': {'AdmEmail1': 1}}])
Results :  [{'_id': ObjectId('67455a282a2b11d96a7953b6'), 'AdmEmail1': 'lkmonroe@acoe.org'}]
####################################################################################################
NL Query :  Extract CharterNum from schools where Charter = 1.


Reconstructed Query :  extract CharterNum schools where Charter = <number>
Parse Tree :  (S
  (CMD extract)
  (COLUMN (CAT CharterNum))
  (TABLE schools)
  (CLAUSE
    where
    (COLUMN (NOMCAT Charter))
    (NOP =)
    (VALUE <number>)
    (CLAUSE )))
SQL :  SELECT CharterNum FROM schools WHERE Charter = 1
NoSQL :  schools.aggregate([{'$match': {'Charter': {'$eq': 1}}}, {'$project': {'CharterNum': 1}}])
Results :  [{'_id': ObjectId('67455a282a2b11d96a7953b7'), 'CharterNum': '0728'}, {'_id': ObjectId('67455a282a2b11d96a7953b8'), 'CharterNum': '0811'}, {'_id': ObjectId('67455a282a2b11d96a7953b9'), 'CharterNum': '1049'}]
####################################################################################################
NL Query :  Select distinct District names from schools.
Reconstructed Query :  select distinct District EdOpsName schools
Parse Tree :  (S
  (CMD select)
  (AGG (DISTINCT distinct))
  (COLUMN (CAT District) (COLUMN (CAT EdOpsName)))
  (CLAUSE )
  (TABLE schools))
SQL :  

Reconstructed Query :  pull unique StatusType schools where State = <string>
Parse Tree :  (S
  (CMD pull)
  (AGG (DISTINCT unique))
  (COLUMN (CAT StatusType))
  (TABLE schools)
  (CLAUSE
    where
    (COLUMN (CAT State))
    (NOP =)
    (VALUE <string>)
    (CLAUSE )))
SQL :  SELECT DISTINCT(StatusType) FROM schools WHERE State = 'CA'
NoSQL :  schools.aggregate([{'$match': {'State': {'$eq': 'CA'}}}, {'$group': {'_id': None, 'result': {'$addToSet': '$StatusType'}}}])
Results :  [{'_id': None, 'result': ['Merged', 'Closed', 'Active', 'Pending']}]


In [6]:
print(tester.cfg)


    S -> CMD "*" TABLE CLAUSE | CMD COLUMN TABLE CLAUSE | CMD AGG COLUMN TABLE CLAUSE | CMD "*" CLAUSE TABLE | CMD COLUMN CLAUSE TABLE| CMD AGG COLUMN CLAUSE TABLE | CLAUSE CMD COLUMN TABLE | CLAUSE CMD AGG COLUMN TABLE | CLAUSE CMD COLUMN TABLE CLAUSE | CLAUSE CMD AGG COLUMN TABLE CLAUSE | TABLE CMD COLUMN CLAUSE | TABLE CMD AGG COLUMN CLAUSE | TABLE CLAUSE CMD COLUMN | TABLE CLAUSE CMD AGG COLUMN | TABLE CLAUSE CMD COLUMN CLAUSE | TABLE CLAUSE CMD AGG COLUMN CLAUSE | CLAUSE CMD COLUMN CLAUSE TABLE | CLAUSE CMD AGG COLUMN CLAUSE TABLE | CLAUSE TABLE CMD COLUMN | CLAUSE TABLE CMD AGG COLUMN | CLAUSE TABLE CMD COLUMN CLAUSE | CLAUSE TABLE CMD AGG COLUMN CLAUSE
    CLAUSE -> "where" COLUMN NOP VALUE CLAUSE | "group" "by" COLUMN CLAUSE | "order" "by" COLUMN CLAUSE 
    CLAUSE -> 
    COLUMN -> CAT COLUMN | NUM COLUMN | NOMCAT COLUMN | ORDCAT COLUMN | CAT AGG COLUMN | NUM AGG COLUMN | NOMCAT AGG COLUMN | ORDCAT AGG COLUMN
    COLUMN -> CAT | NUM | NOMCAT | ORDCAT
    CAT -> "_id" | "AdmEm